In [2]:
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from sqlalchemy import create_engine
import matplotlib as mpl
import numpy as np
import seaborn as sns
from scipy import stats
from sklearn import preprocessing
%matplotlib inline

In [3]:
path = r"XXXX-XXXX"
con = sqlite3.connect(path)  

In [4]:
gr = pd.read_sql_query("SELECT *  FROM goalkeeperRating ORDER BY fixture_id", con)
gr = gr.drop('index', axis=1)

In [5]:
dr = pd.read_sql_query("SELECT * FROM defenderRating ORDER BY fixture_id", con)
dr = dr.drop('index', axis=1)

In [6]:
mr = pd.read_sql_query("SELECT * FROM midfieldRating ORDER BY fixture_id", con)
mr = mr.drop('index', axis=1)

In [7]:
fr = pd.read_sql_query("SELECT * FROM forwardRating ORDER BY fixture_id", con)
fr = fr.drop('index', axis=1)

In [8]:
gr_dr = pd.merge(gr, dr, how='outer', on=['fixture_id', 'player_id'])

In [9]:
gr_dr_mr = pd.merge(gr_dr, mr, how='outer', on=['fixture_id', 'player_id'])

In [10]:
all_players = pd.merge(gr_dr_mr, fr, how='outer', on=['fixture_id', 'player_id'])

In [11]:
col_list= ['g_rating', 'd_rating', 'm_rating', 'f_rating']
# sum specific columns
dfe = all_players[col_list].sum(axis=1)
all_players.insert(2, 'player_rating', dfe)
all_players = all_players.drop('g_rating', axis=1)
all_players = all_players.drop('d_rating', axis=1)
all_players = all_players.drop('m_rating',  axis=1)
all_players = all_players.drop('f_rating', axis=1)
display(all_players)

,fixture_id,player_id,player_rating
0,65,882,-0.347814
1,65,2728,-0.380995
2,66,18886,0.021374
3,66,159,0.401526
4,67,18858,0.234389
...,...,...,...
319776,705682,33320,-0.414302
319777,705683,10361,-0.588891
319778,705683,41930,-0.042132
319779,705683,41188,-0.276957


In [12]:
lineups = pd.read_sql_query("SELECT *  FROM lineups ORDER BY fixture_id", con)
pd.set_option('display.max_rows', 60)
pd.set_option('display.min_rows', 10)
display(lineups)

,fixture_id,team_id,timestamp,player_id,player_name,player_number,player_position,grid
0,65,33,1533927600,882.0,David de Gea,1.0,G,1:1
1,65,33,1533927600,887.0,M. Darmian,36.0,D,2:4
2,65,33,1533927600,889.0,V. Lindelöf,2.0,D,2:3
3,65,33,1533927600,891.0,L. Shaw,23.0,D,2:2
4,65,33,1533927600,885.0,E. Bailly,3.0,D,2:1
...,...,...,...,...,...,...,...,...
322757,705683,240,1622397600,15327.0,Leandro Silva,21.0,M,3:2
322758,705683,240,1622397600,43304.0,Pedro Moreira,20.0,M,3:1
322759,705683,240,1622397600,41930.0,André Bukia,10.0,F,4:3
322760,705683,240,1622397600,41188.0,André Silva,7.0,F,4:2


In [37]:
fixtures = pd.read_sql_query("SELECT *  FROM fixtures ORDER BY fixture_id", con)
pd.set_option('display.max_rows', 60)
pd.set_option('display.min_rows', 10)
display(fixtures)

,fixture_id,referee,timestamp,league_id,league_name,season,league_round,result,home_id,home_name,home_goals,away_id,away_name,away_goals
0,65,"Andre Marriner, England",1533927600,39,Premier League,2018,Regular Season - 1,0.0,33,Manchester United,2.0,46,Leicester,1.0
1,66,"Martin Atkinson, England",1533987000,39,Premier League,2018,Regular Season - 1,1.0,34,Newcastle,1.0,47,Tottenham,2.0
2,67,"Kevin Friend, England",1533996000,39,Premier League,2018,Regular Season - 1,0.0,35,Bournemouth,2.0,43,Cardiff,0.0
3,68,"Mike Dean, England",1533996000,39,Premier League,2018,Regular Season - 1,1.0,36,Fulham,0.0,52,Crystal Palace,2.0
4,69,"Chris Kavanagh, England",1533996000,39,Premier League,2018,Regular Season - 1,1.0,37,Huddersfield,0.0,49,Chelsea,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14666,705637,"Benoît Bastien, France",1622390400,61,Ligue 1,2020,Relegation Round,1.0,83,Nantes,0.0,96,Toulouse,1.0
14667,705680,F. Zwayer,1622046600,78,Bundesliga 1,2020,Relegation Round,1.0,192,FC Koln,0.0,191,Holstein Kiel,1.0
14668,705681,"Deniz Aytekin, Germany",1622304000,78,Bundesliga 1,2020,Relegation Round,1.0,191,Holstein Kiel,1.0,192,FC Koln,5.0
14669,705682,João Pedro Pinheiro,1622061900,94,Primeira Liga,2020,Relegation Round,0.0,240,Arouca,3.0,226,Rio Ave,0.0


In [14]:
#LINEUP hat keine Information über home und away daher werden home lineups rausgesucht
home_lineups = pd.read_sql_query("SELECT *  FROM lineups ORDER BY fixture_id", con)
home_lineups.insert(1, 'home_id', home_lineups.pop('team_id'))
home_lineups = pd.merge(home_lineups, fixtures, how='inner', on=['fixture_id', 'home_id'])
home_lineups = home_lineups[['fixture_id', 'player_id', 'player_name', 'player_position']]
display(home_lineups)

#EXTRAHIERE die fixture_ids
fixture_ids = home_lineups[['fixture_id']]
#159654 mit duplikaten 14514 ohne duplikat
fixture_ids = fixture_ids.drop_duplicates()



#order wird behalten


,fixture_id,player_id,player_name,player_position
0,65,882.0,David de Gea,G
1,65,887.0,M. Darmian,D
2,65,889.0,V. Lindelöf,D
3,65,891.0,L. Shaw,D
4,65,885.0,E. Bailly,D
...,...,...,...,...
159649,705683,41183.0,Filipe Augusto,M
159650,705683,41176.0,Fábio Coentrão,M
159651,705683,3325.0,Gelson Dala,M
159652,705683,24860.0,Carlos Mané,M


In [ ]:
#ERSTELLE zwei listen mit fixture_id und Spielern-Home
def check_for_next_fixture_id(current_fixture, next_fixture, i, all_players, home_lineups):
    if(len(next_fixture)==0):
        next_fixture = all_players.loc[(all_players["fixture_id"] == home_lineups.iloc[i+1]['fixture_id']) & (all_players["player_id"] == home_lineups.iloc[i+1]['player_id'])]
        next_fixture_id = check_for_next_fixture_id(current_fixture, next_fixture, i+1, all_players, home_lineups)
        return next_fixture_id
    else:
        next_fixture_id = next_fixture.iloc[0]['fixture_id']
        return next_fixture_id

def check_for_next_fixture_iterator(current_fixture, next_fixture, i, all_players, home_lineups):
    if(len(next_fixture)==0):
        next_fixture = all_players.loc[(all_players["fixture_id"] == home_lineups.iloc[i+1]['fixture_id']) & (all_players["player_id"] == home_lineups.iloc[i+1]['player_id'])]
        a = check_for_next_fixture_iterator(current_fixture, next_fixture, i+1, all_players, home_lineups)
        return a
    else:
        a = i-1
        return a


def loc_player_rating(player):
    if(len(player)==0):
        return 0.0
    else:
        return player.iloc[0]['player_id']



data = []
player_ratings = []
fixture_ids_list = []
for i in range(0, len(home_lineups)): #len(home_lineups)
    player = all_players.loc[(all_players["fixture_id"] == home_lineups.iloc[i]['fixture_id']) & (all_players["player_id"] == home_lineups.iloc[i]['player_id'])]
    data.append(loc_player_rating(player))
    if(i+1 == len(home_lineups)):
        player_ratings.append(data)
        fixture_ids_list.append(home_lineups.iloc[i]['fixture_id'])
        data = []
    else:
        next_fixture = all_players.loc[(all_players["fixture_id"] == home_lineups.iloc[i+1]['fixture_id']) & (all_players["player_id"] == home_lineups.iloc[i+1]['player_id'])]
        next_fixture_id = check_for_next_fixture_id(home_lineups.iloc[i]['fixture_id'], next_fixture, i+1, all_players, home_lineups)
        i_new = check_for_next_fixture_iterator(home_lineups.iloc[i]['fixture_id'], next_fixture, i+1, all_players, home_lineups)
        if(home_lineups.iloc[i]['fixture_id'] != next_fixture_id):
            print(i_new)
            i = i_new
            print(i)
            player_ratings.append(data)
            fixture_ids_list.append(home_lineups.iloc[i]['fixture_id'])
            data = []

print(len(player_ratings))
print(len(fixture_ids_list))
#3638, 3639, 3640, 3640, 3640, 3641, 3642,


In [111]:
df_fx = pd.DataFrame(fixture_ids_list, columns=['fixture_id'])
#display(df_fx)
df_pl = pd.DataFrame(player_ratings)
df_pl.insert(0, 'fixture_id', df_fx['fixture_id'])
# df_pl = df_pl[df_pl.isna().any(axis=1)]
df_pl = df_pl.drop_duplicates()
df_pl = df_pl.fillna(0)
#display(df_pl)
df_pl.groupby(['fixture_id']).count()
column_names = [0,1,2,3,4,5,6,7,8,9,10]
df_pl['Fruit Total']= df_pl[column_names].sum(axis=1)
df= df_pl[df_pl['Fruit Total'] != 0]
pd.set_option('display.max_rows', 90)
pd.set_option('display.min_rows', 10)
df = df.drop_duplicates()
df.groupby(['fixture_id'])
df = df.rename({0: "h_0", 1: "h_1", 2: "h_2", 3: "h_3", 4: "h_4", 5: "h_5",6: "h_6", 7: "h_7", 8: "h_8",9: "h_9", 10: "h_10"}, axis=1)
df = df.drop(df[['Fruit Total']], axis=1)
display(df)
#df.groupby('Fruit Total').filter(lambda x : len(x)>3)

#df = df_pl_na.loc[:, (df_pl_na != 0).any(axis=0)]
# column_names = ['0','1','2','3','4','5','6','7','8', '9', '10']

,fixture_id,h_0,h_1,h_2,h_3,h_4,h_5,h_6,h_7,h_8,h_9,h_10
0,65,882.0,887.0,889.0,891.0,885.0,901.0,904.0,905.0,899.0,910.0,909.0
1,66,18886.0,18891.0,18894.0,18892.0,18897.0,18904.0,18898.0,18903.0,18902.0,18907.0,18906.0
2,67,18858.0,18864.0,18869.0,18863.0,18861.0,18879.0,18874.0,18873.0,18870.0,2939.0,18881.0
3,68,19011.0,19019.0,19017.0,19015.0,19016.0,19029.0,19031.0,19025.0,3243.0,19032.0,2825.0
4,69,19038.0,2731.0,19044.0,19045.0,19043.0,19042.0,19049.0,19050.0,19051.0,2734.0,3395.0
...,...,...,...,...,...,...,...,...,...,...,...,...
14815,705637,30393.0,1407.0,20545.0,21088.0,21091.0,21104.0,21100.0,93671.0,2781.0,21497.0,3344.0
14816,705680,24785.0,24794.0,1408.0,24789.0,24792.0,24807.0,21587.0,26.0,25349.0,158121.0,24801.0
14817,705681,25570.0,25206.0,24961.0,26121.0,26453.0,24965.0,25319.0,24967.0,2906.0,26525.0,25167.0
14818,705682,96687.0,41906.0,41609.0,95252.0,157284.0,41930.0,15327.0,43304.0,41139.0,41435.0,41188.0


In [69]:

fck = df_pl.loc[(df_pl["fixture_id"] == 201813)]
display(fck)
home_fck = all_players.loc[(all_players["fixture_id"] == 201813)]
display(home_fck)

,fixture_id,0,1,2,3,4,5,6,7,8,9,10
10302,201813,41229.0,42988.0,104408.0,15341.0,41493.0,41178.0,104448.0,90546.0,41308.0,41224.0,NaN
10303,201813,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10304,201813,0.0,49244.0,41908.0,41266.0,41203.0,41421.0,41107.0,11006.0,41391.0,41282.0,NaN
10305,201813,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


,fixture_id,player_id,player_rating
108462,201813,49244,0.261828
108463,201813,41908,-0.474467
108464,201813,41266,-0.167078
108465,201813,41203,-0.307071
108466,201813,96501,0.018759
108467,201813,41488,-0.062311
225923,201813,41421,0.076364
225924,201813,41107,-0.068030
225925,201813,11006,-0.018569
225926,201813,41391,-0.506081


In [90]:
#LINEUP hat keine Information über home und away daher werden away lineups rausgesucht
away_lineups = pd.read_sql_query("SELECT *  FROM lineups ORDER BY fixture_id", con)
away_lineups.insert(1, 'away_id', away_lineups.pop('team_id'))
away_lineups = pd.merge(away_lineups, fixtures, how='inner', on=['fixture_id', 'away_id'])
display(away_lineups)

#order wird behalten

,fixture_id,away_id,timestamp_x,player_id,player_name,player_number,player_position,grid,referee,timestamp_y,league_id,league_name,season,league_round,result,home_id,home_name,home_goals,away_name,away_goals
0,65,46,1533927600,2728.0,K. Schmeichel,1.0,G,1:1,"Andre Marriner, England",1533927600,39,Premier League,2018,Regular Season - 1,0.0,33,Manchester United,2.0,Leicester,1.0
1,65,46,1533927600,18774.0,W. Morgan,5.0,D,2:4,"Andre Marriner, England",1533927600,39,Premier League,2018,Regular Season - 1,0.0,33,Manchester United,2.0,Leicester,1.0
2,65,46,1533927600,2935.0,H. Maguire,15.0,D,2:3,"Andre Marriner, England",1533927600,39,Premier League,2018,Regular Season - 1,0.0,33,Manchester United,2.0,Leicester,1.0
3,65,46,1533927600,18771.0,Ricardo Pereira,14.0,M,2:2,"Andre Marriner, England",1533927600,39,Premier League,2018,Regular Season - 1,0.0,33,Manchester United,2.0,Leicester,1.0
4,65,46,1533927600,2933.0,B. Chilwell,3.0,D,2:1,"Andre Marriner, England",1533927600,39,Premier League,2018,Regular Season - 1,0.0,33,Manchester United,2.0,Leicester,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
159649,705683,240,1622397600,15327.0,Leandro Silva,21.0,M,3:2,"Hugo Miguel, Portugal",1622397600,94,Primeira Liga,2020,Relegation Round,1.0,226,Rio Ave,0.0,Arouca,2.0
159650,705683,240,1622397600,43304.0,Pedro Moreira,20.0,M,3:1,"Hugo Miguel, Portugal",1622397600,94,Primeira Liga,2020,Relegation Round,1.0,226,Rio Ave,0.0,Arouca,2.0
159651,705683,240,1622397600,41930.0,André Bukia,10.0,F,4:3,"Hugo Miguel, Portugal",1622397600,94,Primeira Liga,2020,Relegation Round,1.0,226,Rio Ave,0.0,Arouca,2.0
159652,705683,240,1622397600,41188.0,André Silva,7.0,F,4:2,"Hugo Miguel, Portugal",1622397600,94,Primeira Liga,2020,Relegation Round,1.0,226,Rio Ave,0.0,Arouca,2.0


In [ ]:
#ERSTELLE zwei listen mit fixture_id und Spielern-away
def check_for_next_fixture_id(current_fixture, next_fixture, i, all_players, away_lineups):
    if(len(next_fixture)==0):
        next_fixture = all_players.loc[(all_players["fixture_id"] == away_lineups.iloc[i+1]['fixture_id']) & (all_players["player_id"] == away_lineups.iloc[i+1]['player_id'])]
        next_fixture_id = check_for_next_fixture_id(current_fixture, next_fixture, i+1, all_players, away_lineups)
        return next_fixture_id
    else:
        next_fixture_id = next_fixture.iloc[0]['fixture_id']
        return next_fixture_id

def check_for_next_fixture_iterator(current_fixture, next_fixture, i, all_players, away_lineups):
    if(len(next_fixture)==0):
        next_fixture = all_players.loc[(all_players["fixture_id"] == away_lineups.iloc[i+1]['fixture_id']) & (all_players["player_id"] == away_lineups.iloc[i+1]['player_id'])]
        a = check_for_next_fixture_iterator(current_fixture, next_fixture, i+1, all_players, away_lineups)
        return a
    else:
        a = i-1
        return a


def loc_player_rating(player):
    if(len(player)==0):
        return 0.0
    else:
        return player.iloc[0]['player_id']



data = []
away_player_ratings = []
away_fixture_ids_list = []
for i in range(0, len(away_lineups)): #len(away_lineups)
    player = all_players.loc[(all_players["fixture_id"] == away_lineups.iloc[i]['fixture_id']) & (all_players["player_id"] == away_lineups.iloc[i]['player_id'])]
    data.append(loc_player_rating(player))
    if(i+1 == len(away_lineups)):
        away_player_ratings.append(data)
        away_fixture_ids_list.append(away_lineups.iloc[i]['fixture_id'])
        data = []
    else:
        next_fixture = all_players.loc[(all_players["fixture_id"] == away_lineups.iloc[i+1]['fixture_id']) & (all_players["player_id"] == away_lineups.iloc[i+1]['player_id'])]
        next_fixture_id = check_for_next_fixture_id(away_lineups.iloc[i]['fixture_id'], next_fixture, i+1, all_players, away_lineups)
        i_new = check_for_next_fixture_iterator(away_lineups.iloc[i]['fixture_id'], next_fixture, i+1, all_players, away_lineups)
        if(away_lineups.iloc[i]['fixture_id'] != next_fixture_id):
            print(i_new)
            i = i_new
            print(i)
            away_player_ratings.append(data)
            away_fixture_ids_list.append(away_lineups.iloc[i]['fixture_id'])
            data = []

print(len(away_player_ratings))
print(len(away_fixture_ids_list))
#3638, 3639, 3640, 3640, 3640, 3641, 3642,


In [109]:
df_fx = pd.DataFrame(away_fixture_ids_list, columns=['fixture_id'])
#display(df_fx)
df_pl = pd.DataFrame(away_player_ratings)
df_pl.insert(0, 'fixture_id', df_fx['fixture_id'])
# df_pl = df_pl[df_pl.isna().any(axis=1)]
df_pl = df_pl.drop_duplicates()
df_pl = df_pl.fillna(0)
#display(df_pl)
df_pl.groupby(['fixture_id']).count()
column_names = [0,1,2,3,4,5,6,7,8,9,10]
df_pl['Fruit Total']= df_pl[column_names].sum(axis=1)
away_df= df_pl[df_pl['Fruit Total'] != 0]
pd.set_option('display.max_rows', 90)
pd.set_option('display.min_rows', 10)
away_df = away_df.drop_duplicates()
away_df.groupby(['fixture_id'])
away_df = away_df.rename({0: "a_0", 1: "a_1", 2: "a_2", 3: "a_3", 4: "a_4", 5: "a_5",6: "a_6", 7: "a_7", 8: "a_8",9: "a_9", 10: "a_10"}, axis=1)
away_df = away_df.drop(away_df[['Fruit Total']], axis=1)
display(away_df)

,fixture_id,a_0,a_1,a_2,a_3,a_4,a_5,a_6,a_7,a_8,a_9,a_10
0,65,2728.0,18774.0,2935.0,18771.0,2933.0,114.0,3421.0,18786.0,18781.0,18784.0,2778.0
1,66,159.0,170.0,163.0,164.0,168.0,179.0,174.0,178.0,172.0,175.0,184.0
2,67,18983.0,18994.0,18991.0,18986.0,18992.0,18987.0,18993.0,19004.0,19003.0,19007.0,19008.0
3,68,18836.0,18842.0,18844.0,18845.0,18843.0,18846.0,18849.0,18854.0,18852.0,2928.0,3247.0
4,69,2273.0,2283.0,2280.0,2278.0,2285.0,2287.0,2289.0,2290.0,2294.0,2299.0,59.0
...,...,...,...,...,...,...,...,...,...,...,...,...
14838,705637,21079.0,22136.0,1928.0,39038.0,22134.0,8482.0,129682.0,22147.0,2483.0,1134.0,19006.0
14839,705680,25570.0,25206.0,24961.0,26453.0,24960.0,25236.0,25319.0,2906.0,24800.0,26525.0,24973.0
14840,705681,24785.0,36916.0,1408.0,24789.0,24791.0,21587.0,24801.0,26.0,25349.0,24803.0,2862.0
14841,705682,46811.0,19065.0,41172.0,44419.0,26767.0,26779.0,3445.0,41183.0,24860.0,3325.0,33320.0


In [112]:
display(df)

,fixture_id,h_0,h_1,h_2,h_3,h_4,h_5,h_6,h_7,h_8,h_9,h_10
0,65,882.0,887.0,889.0,891.0,885.0,901.0,904.0,905.0,899.0,910.0,909.0
1,66,18886.0,18891.0,18894.0,18892.0,18897.0,18904.0,18898.0,18903.0,18902.0,18907.0,18906.0
2,67,18858.0,18864.0,18869.0,18863.0,18861.0,18879.0,18874.0,18873.0,18870.0,2939.0,18881.0
3,68,19011.0,19019.0,19017.0,19015.0,19016.0,19029.0,19031.0,19025.0,3243.0,19032.0,2825.0
4,69,19038.0,2731.0,19044.0,19045.0,19043.0,19042.0,19049.0,19050.0,19051.0,2734.0,3395.0
...,...,...,...,...,...,...,...,...,...,...,...,...
14815,705637,30393.0,1407.0,20545.0,21088.0,21091.0,21104.0,21100.0,93671.0,2781.0,21497.0,3344.0
14816,705680,24785.0,24794.0,1408.0,24789.0,24792.0,24807.0,21587.0,26.0,25349.0,158121.0,24801.0
14817,705681,25570.0,25206.0,24961.0,26121.0,26453.0,24965.0,25319.0,24967.0,2906.0,26525.0,25167.0
14818,705682,96687.0,41906.0,41609.0,95252.0,157284.0,41930.0,15327.0,43304.0,41139.0,41435.0,41188.0


In [116]:
player_ratings_df = pd.merge(df, away_df, how='inner', on=['fixture_id'])
display(player_ratings_df)


,fixture_id,h_0,h_1,h_2,h_3,h_4,h_5,h_6,h_7,h_8,...,a_1,a_2,a_3,a_4,a_5,a_6,a_7,a_8,a_9,a_10
0,65,882.0,887.0,889.0,891.0,885.0,901.0,904.0,905.0,899.0,...,18774.0,2935.0,18771.0,2933.0,114.0,3421.0,18786.0,18781.0,18784.0,2778.0
1,66,18886.0,18891.0,18894.0,18892.0,18897.0,18904.0,18898.0,18903.0,18902.0,...,170.0,163.0,164.0,168.0,179.0,174.0,178.0,172.0,175.0,184.0
2,67,18858.0,18864.0,18869.0,18863.0,18861.0,18879.0,18874.0,18873.0,18870.0,...,18994.0,18991.0,18986.0,18992.0,18987.0,18993.0,19004.0,19003.0,19007.0,19008.0
3,68,19011.0,19019.0,19017.0,19015.0,19016.0,19029.0,19031.0,19025.0,3243.0,...,18842.0,18844.0,18845.0,18843.0,18846.0,18849.0,18854.0,18852.0,2928.0,3247.0
4,69,19038.0,2731.0,19044.0,19045.0,19043.0,19042.0,19049.0,19050.0,19051.0,...,2283.0,2280.0,2278.0,2285.0,2287.0,2289.0,2290.0,2294.0,2299.0,59.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14448,705637,30393.0,1407.0,20545.0,21088.0,21091.0,21104.0,21100.0,93671.0,2781.0,...,22136.0,1928.0,39038.0,22134.0,8482.0,129682.0,22147.0,2483.0,1134.0,19006.0
14449,705680,24785.0,24794.0,1408.0,24789.0,24792.0,24807.0,21587.0,26.0,25349.0,...,25206.0,24961.0,26453.0,24960.0,25236.0,25319.0,2906.0,24800.0,26525.0,24973.0
14450,705681,25570.0,25206.0,24961.0,26121.0,26453.0,24965.0,25319.0,24967.0,2906.0,...,36916.0,1408.0,24789.0,24791.0,21587.0,24801.0,26.0,25349.0,24803.0,2862.0
14451,705682,96687.0,41906.0,41609.0,95252.0,157284.0,41930.0,15327.0,43304.0,41139.0,...,19065.0,41172.0,44419.0,26767.0,26779.0,3445.0,41183.0,24860.0,3325.0,33320.0


In [117]:
engine = create_engine(r'XXXX-XXXX', echo=False)
player_ratings_df.to_sql('fixtureAllPlayerRatings', con=engine, if_exists='fail') #if_exists='append'